In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Definición de constantes y parámetros del problema

In [ ]:
c = 299792458.0

f0 = 1e9
er1 = 1
er2 = 16.0
ert = 4.0

eps0 = 8.854e-12
eps1 = 8.854e-12
eps2 = er1 * eps0

eta0 = 120 * np.pi
eta1 = eta0 / np.sqrt(er1)  
eta2 = eta0 / np.sqrt(er2)  
etat = eta0 / np.sqrt(ert)

# Lambda en la lámina a f0
lambda_m0 = c / (f0 * np.sqrt(eps_m / eps0))

# Espesor de la lámina (cuarto de onda)
d = lambda_m0 / 4


# Parte a

# Parte b